In [ ]:
unique_nuclei = df['Atom'].unique()

# Define number of subplots
num_nuclei = len(unique_nuclei)

# Create subplots
fig, axs = plt.subplots(num_nuclei, 1, figsize=(8, 4 * num_nuclei))

# If only one nucleus, axs is not a list, so convert it to list for consistency
if num_nuclei == 1:
    axs = [axs]
    
def remove_prefix(molecule_name, prefix="opt_disp_struct_"):
    return molecule_name.replace(prefix, '')
    
# Loop through each unique nucleus and create a plot
for i, nucleus in enumerate(unique_nuclei):
    # Filter data for the current nucleus
    nucleus_data = df[df['Atom'] == nucleus]
    
    molecule_suffixes = [remove_prefix(molecule) for molecule in nucleus_data['Molecule']]
    
    # Plot sigma_iso against molecule index or name
    axs[i].plot(molecule_suffixes, nucleus_data['sigma_iso'], marker='o', color='black', label='$\sigma$_iso')
    
    axs[i].plot(molecule_suffixes, nucleus_data['sigma_xx'], marker='d', color='gold', label='$\sigma$_xx')
    axs[i].plot(molecule_suffixes, nucleus_data['sigma_yy'], marker='*', color='cyan', label='$\sigma$_yy')
    axs[i].plot(molecule_suffixes, nucleus_data['sigma_zz'], marker='+', color='violet', label='$\sigma$_xx')

    # Set labels and title
    axs[i].set_title(f'{nucleus}')
    axs[i].set_xlabel('benzene_h2o')
    axs[i].set_ylabel('$\Delta$$\sigma$')
    
    axs[i].set_xticks(range(len(molecule_suffixes)))
    axs[i].set_xticklabels(molecule_suffixes, rotation=90, ha="right")
       
    axs[i].legend()

plt.tight_layout()

plt.show()

In [ ]:
# --- Application example 2) J Coupling Tensor --- #

# Initialize an empty list to collect data
datapw = []


# Loop through the number of jobs and process each molecule
for job_number in range(n_jobs):
    molecule_name = list_molecules[job_number]
    
    # Extract the distance dictionary for the current job
    distance_dic = r_AB[job_number]
    
    # Check if the distance dictionary is valid
    if isinstance(distance_dic, dict):
        # Loop through each pair and its distance
        for (atom1, atom2), distance in distance_dic.items():
            # Create a row of data for this pair
            row_datapw = {
                'Molecule': molecule_name,
                'Atom_1': atom1,
                'Atom_2': atom2,
                'r_12': distance,
                'x_coord_1' : None, 
                'y_coord_1' : None, 
                'z_coord_1' : None, 
                'x_coord_2' : None, 
                'y_coord_2' : None, 
                'z_coord_2' : None,
                'J_iso' : None,
                'J_FC_11' : None,
                'J_FC_22' : None,
                'J_FC_33' : None,
                'J_DSO_11' : None,
                'J_DSO_22' : None,
                'J_DSO_33' : None,
                'J_PSO_11' : None,
                'J_PSO_22' : None,
                'J_PSO_33' : None,
                'J_SD_11' : None,
                'J_SD_22' : None,
                'J_SD_33' : None,
                'Mayer_BO' : None
            }
            # Append the row to the list
            datapw.append(row_datapw)

    m_bo_dict = mayer_bo[job_number]
        
    if isinstance(m_bo_dict, dict):
        for atom1, pairs in m_bo_dict.items():
            for atom2, bond_order in pairs:
                # Create a row of data for Mayer BO
                row_datapw = {
                    'Mayer_BO': bond_order
                }
                # Append the row to the Mayer BO list
                datapw.append(row_mayer_bo)

# Convert the list of rows to a DataFrame
datadf = pd.DataFrame(datapw)

# Load the existing DataFrame if needed (make sure to define `nucprop_df` earlier)
# Example: nucprop_df = pd.read_csv('existing_data.csv')

# Concatenate with the existing DataFrame
pw_nucprop_df2 = pd.concat([pw_nucprop_df, datadf], ignore_index=True)

# Save the updated DataFrame to a CSV file
#w_nucprop_df.to_csv('updated_nuclear_pairs.csv', index=False)

# Display the updated DataFrame
pw_nucprop_df2